### FYI, 

You can grab the full roster of students AND what group they're in from Canvas (go to People => Project Groups => Import)

Using this script for pulling the project groups from Canvas, pulling the guthub IDs from a Canvas Quiz (manually), combining them to create a list of groups with respective github IDs, isolating the non-assigned students (use full roster), and create repos in Github for each group and send invites. 

In [1]:
import pandas as pd
import requests
import time

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

### Get all enrolled students from Canvas gradebook

In [3]:
# pull all students using gradebook from Canvas
enrolled_students = pd.read_csv('2022-10-12T0952_Grades-COGS108_FA22_A00.csv')

In [4]:
enrolled_students

,Student,ID,SIS User ID,SIS Login ID,Section,D1 (531421),D2 (531422),Practice Assignment (531444),A1 (531416),Q1 (531409),...,Project Final Score,Project Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score,Current Grade,Unposted Current Grade,Final Grade,Unposted Final Grade
0,Points Possible,NaN,NaN,NaN,NaN,2.0,2.0,1.0,8.0,1.00,...,(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only)
1,"Abraham, Rita",64598.0,A15892063,r2abraha,COGS 108 - A03 [87921],NaN,NaN,NaN,NaN,0.90,...,NaN,NaN,90.00,90.00,12.63,12.63,A-,A-,F,F
2,"Ahmed, Tonoya",108059.0,A16672415,ttahmed,COGS 108 - A07 [87925],NaN,NaN,NaN,NaN,1.00,...,NaN,NaN,100.00,100.00,14.04,14.04,A+,A+,F,F
3,"Aimer-Ghani, Ryley",138729.0,A17169786,raimerghani,COGS 108 - A03 [87921],NaN,NaN,NaN,NaN,0.88,...,NaN,NaN,87.50,87.50,12.63,12.63,B+,B+,F,F
4,"Alcaraz-Andrade, Gissel",143171.0,A17207543,galcarazandrade,COGS 108 - A05 [87923],NaN,NaN,NaN,NaN,1.00,...,NaN,NaN,100.00,100.00,14.04,14.04,A+,A+,F,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,"Zhou, Zhengyuan",65764.0,A16146369,zzhou,COGS 108 - A03 [87921],NaN,NaN,NaN,NaN,0.90,...,NaN,NaN,90.00,90.00,12.63,12.63,A-,A-,F,F
441,"Zhu, Yilin",69440.0,A15862322,yiz095,COGS 108 - A02 [87920],NaN,NaN,NaN,NaN,0.98,...,NaN,NaN,97.50,97.50,13.68,13.68,A+,A+,F,F
442,"Zou, Ellie",129572.0,A17117056,w3zou,COGS 108 - A07 [87925],NaN,NaN,NaN,NaN,1.00,...,NaN,NaN,100.00,100.00,14.04,14.04,A+,A+,F,F
443,"Zwiebel, Jennifer",104054.0,A16324852,jzwiebel,COGS 108 - A05 [87923],NaN,NaN,NaN,NaN,1.00,...,NaN,NaN,100.00,100.00,14.04,14.04,A+,A+,F,F


In [5]:
enrolled_students = enrolled_students.drop([0,444]).reset_index(drop=True)[['Student','SIS User ID','SIS Login ID']]
enrolled_students = enrolled_students.rename( {'SIS User ID':'SID', 'SIS Login ID':'email'},axis=1)
# turn last, first into first last
enrolled_students['Student'] = ( enrolled_students['Student']
                                .apply( lambda x: x.split(', ')[::-1]) # reverse order around the ,
                                .apply(lambda x: x[0]+' '+x[1]) # join with space between
                               )
enrolled_students = enrolled_students.dropna() # drop the canvas test student with NaN SID
enrolled_students = enrolled_students[enrolled_students.Student != 'Cogs 108 Grader Account'] 
enrolled_students.Student.value_counts()

Jason Lee           2
Rita Abraham        1
Yicheng Qu          1
Shay Samat          1
Josh Salce          1
                   ..
Brandon Kao         1
Seung Kang          1
Annelise Kamm       1
Sarah Kagzi         1
Jennifer Zwiebel    1
Name: Student, Length: 441, dtype: int64

### Import data for github IDs (manually from Canvas)

# How to grab github IDs

```
1) Go to Canvas quiz for github IDs
2) Go to "Quiz Statistics", then click on "Student Analysis"
3) This downloads a CSV. Move the CSV to the directory of this notebook. 
4) Run the following cell
```

```
githubs = pd.read_csv('Github user ID Survey Student Analysis Report.csv')
githubs.columns = ['name', 'id', 'sis_id', 'section', 'section_id', 'section_sis_id', 'submitted', 'attempt', 'github', '1.0', 'n correct', 'n incorrect', 'score']
githubs = githubs[['name','sis_id','attempt', 'github']]
#githubs.set_axis(['name', 'sis_id', 'github'], axis=1, inplace=True)
```

#### Clean Github Quiz

In [8]:
# Clean github usernames (WTF happened here ugh)
# This has been done for Fall 22, but you will probably need to do this for future quarters

#githubs.drop_duplicates(subset=['sis_id'], keep='first', inplace=True)
#githubs.to_csv('Github user ID Survey Student Analysis Report_fixed.csv')

In [10]:
# Some REGEX stuff here for cleaning
# will need removal of links, removal of new lines, removal of quotes, other shit
# => I did this manually in excel

In [9]:
githubs = pd.read_csv('Github user ID Survey Student Analysis Report_fixed.csv')

In [10]:
githubs

,Unnamed: 0,name,sis_id,attempt,github
0,0,Kimberly Wong,A16352518,1,kimiiwong
1,1,Brandon Tran,A15206559,1,btran206
2,2,Tonoya Ahmed,A16672415,1,tonoyaahmed46
3,3,Brandon Luong,A17165655,1,BrandonLuong
4,4,Yutong Zhang,A15632165,1,Skyler2727
...,...,...,...,...,...
402,455,Leon Tan,A17185988,1,leontan29
403,456,Baraa Zekeria,A17137018,1,bzekeria
404,459,Tina Nguyen,A15958724,1,tin001ucsd
405,461,Haoyuan Wang,A16162507,1,LukeWang777


Add githubs to enrolled_students AND check which students don't have a github ID

In [11]:
no_gits = []
def find_git(sid):
        try:
            ret = githubs.set_index('sis_id').loc[sid,'github']
        except KeyError:
            print('No github found for: ' + sid)
            no_gits.append(sid)
            ret = None
        return ret

enrolled_students['github'] = enrolled_students.apply(lambda row : find_git(row['SID']), axis = 1)

No github found for: A16138872
No github found for: A15633379
No github found for: A15642345
No github found for: A16131276
No github found for: A15986849
No github found for: A16118006
No github found for: A16503456
No github found for: A16373388
No github found for: A16681107
No github found for: A16946426
No github found for: A17169808
No github found for: A17239984
No github found for: A16410258
No github found for: A16121771
No github found for: A15967880
No github found for: A16526598
No github found for: A15832700
No github found for: A16306812
No github found for: A16394707
No github found for: U09486781
No github found for: A17215453
No github found for: A17210415
No github found for: A15951316
No github found for: A15834896
No github found for: A16416168
No github found for: A15975998
No github found for: A15895435
No github found for: A17151720
No github found for: A16232780
No github found for: A17135615
No github found for: A15936338
No github found for: A16166274
No githu

In [12]:
len(no_gits)

35

In [13]:
# ^That's perfect => nobody is left out!

In [14]:
get_git = enrolled_students[enrolled_students['SID'].isin(no_gits)]

In [15]:
# theses are the students to email (copy/paste below into TO: line) 
# ask them to fill out the Github Quiz ASAP!
git_emails = get_git.email + '@ucsd.edu'

In [16]:
git_emails.to_list()

['lanson@ucsd.edu',
 'tochang@ucsd.edu',
 'ddavilag@ucsd.edu',
 'gdungca@ucsd.edu',
 'q6gong@ucsd.edu',
 'jguiman@ucsd.edu',
 'jhandral@ucsd.edu',
 'ehsiao@ucsd.edu',
 'ejin@ucsd.edu',
 'arkhanna@ucsd.edu',
 'nik001@ucsd.edu',
 'a4ko@ucsd.edu',
 'akourjan@ucsd.edu',
 'jlawler@ucsd.edu',
 'hgle@ucsd.edu',
 'yul145@ucsd.edu',
 'grliu@ucsd.edu',
 'k1luong@ucsd.edu',
 'nemarcus@ucsd.edu',
 'ax008364@ucsd.edu',
 'ximu@ucsd.edu',
 'aparashar@ucsd.edu',
 'jsather@ucsd.edu',
 'rsedano@ucsd.edu',
 'pshahabi@ucsd.edu',
 'isikdar@ucsd.edu',
 'aslater@ucsd.edu',
 'jis036@ucsd.edu',
 'jmvillal@ucsd.edu',
 'tcwhite@ucsd.edu',
 'juw008@ucsd.edu',
 'y1xiao@ucsd.edu',
 'deyoo@ucsd.edu',
 'jiz088@ucsd.edu',
 'jiz081@ucsd.edu']

## pull groups from Canvas group signup (script)

In [17]:
# Note: Accessing the API requires a token. You can generate a token by following these steps
# this token will expire and you will need to regenerate for future quarters

'''
1) Go to your Canvas
2) Go to "Account" => Settings"
3) Scroll to "Approved Integration" and click "+ New Access Token"
4) Save your token (the below token will be expired)
'''
token = '13171~9B7RkWuHE17yFaxPsH2HuaqtBbmDvSCMfYPBVCM5KH9OPab3nQAC3b8XJZ09fuap'

In [18]:
# Make API calls to Canvas API (using requests)

# First get all the groups (including Group Ids)
#TODO: fix API request limit (handling pagination)

raw_groups = []
header = {'Authorization': 'Bearer ' + token}

call = requests.get('https://ucsd.instructure.com:443/api/v1/courses/39457/groups', headers = header)
print(call.status_code)

raw = call.json()
for g in raw:
    raw_groups.append(g)

200


In [19]:
while call.links['current']['url'] != call.links['last']['url']: # repeat API calls until current and last match

    call = requests.get(call.links['next']['url'], headers = header)
    print(call.status_code)

    raw = call.json()
    for g in raw:
        raw_groups.append(g)

200
200
200
200
200
200
200
200
200
200
200
200


In [20]:
raw_groups

[{'id': 101918,
  'name': 'Project Group 1',
  'created_at': '2022-10-08T07:41:06Z',
  'max_membership': 5,
  'is_public': False,
  'join_level': 'invitation_only',
  'group_category_id': 13959,
  'description': None,
  'members_count': 5,
  'storage_quota_mb': 1024,
  'context_type': 'Course',
  'course_id': 39457,
  'avatar_url': None,
  'role': None,
  'leader': None,
  'has_submission': False,
  'concluded': False},
 {'id': 101919,
  'name': 'Project Group 2',
  'created_at': '2022-10-08T07:41:06Z',
  'max_membership': 5,
  'is_public': False,
  'join_level': 'invitation_only',
  'group_category_id': 13959,
  'description': None,
  'members_count': 5,
  'storage_quota_mb': 1024,
  'context_type': 'Course',
  'course_id': 39457,
  'avatar_url': None,
  'role': None,
  'leader': None,
  'has_submission': False,
  'concluded': False},
 {'id': 101920,
  'name': 'Project Group 3',
  'created_at': '2022-10-08T07:41:06Z',
  'max_membership': 5,
  'is_public': False,
  'join_level': 'invit

In [21]:
# Then parse through the json for each group id and get the members and add them to a dataframe
Groups = list()
for group in raw_groups:
    
    # get group information
    g_ID = str(group['id'])
    g_Name = str(group['name'])
    g_count = str(group['members_count'])
    
    # skip empty groups
    if (int(g_count) == 0):
        continue
    
    # make API call to get members of this group
    try: 
        members = requests.get('https://ucsd.instructure.com:443/api/v1/groups/' + g_ID + '/users', headers = {'Authorization': 'Bearer ' + token})
    except: 
        print('Error in pulling members from group: ' + g_Name)
        continue

    members = members.json()
    m_names = []
    m_sids = []
    m_gits = []
    
    # loop through all members of group
    for m in members:
        m_names.append(m['name'])
        sid = m['sis_user_id']
        m_sids.append(sid)
        m_gits.append(enrolled_students.set_index('SID').loc[sid,'github'])
    
    Groups.append([g_Name, m_names, m_sids, m_gits, g_count])
    print('Finished adding group: ' + g_Name)
    
print('\nAll Groups Done!')

Finished adding group: Project Group 1
Finished adding group: Project Group 2
Finished adding group: Project Group 3
Finished adding group: Project Group 4
Finished adding group: Project Group 5
Finished adding group: Project Group 6
Finished adding group: Project Group 7
Finished adding group: Project Group 8
Finished adding group: Project Group 9
Finished adding group: Project Group 10
Finished adding group: Project Group 11
Finished adding group: Project Group 12
Finished adding group: Project Group 13
Finished adding group: Project Group 14
Finished adding group: Project Group 15
Finished adding group: Project Group 16
Finished adding group: Project Group 17
Finished adding group: Project Group 18
Finished adding group: Project Group 19
Finished adding group: Project Group 20
Finished adding group: Project Group 21
Finished adding group: Project Group 22
Finished adding group: Project Group 23
Finished adding group: Project Group 24
Finished adding group: Project Group 25
Finished 

In [22]:
Groups = pd.DataFrame(Groups, columns=['Canvas Group Name', 'names', 'sids', 'gitids', 'n_people'])

In [23]:
def name_change(g_Name):
    number = str(int(g_Name[14:])).zfill(3)
    g_Name = g_Name[:14] + number
    return g_Name

Groups['Canvas Group Name'] = Groups['Canvas Group Name'].apply(lambda x: name_change(x))

In [24]:
Groups['n_people'] = pd.to_numeric(Groups['n_people'])

In [25]:
Groups

,Canvas Group Name,names,sids,gitids,n_people
0,Project Group 001,"[Zilu Ji, Laurensius Junaidy, Kendra Lee, Thy ...","[A16608450, A16617195, A16282947, A16570913, A...","[LUsamiLU, LaurensiusJunaidy, kkbunny247, thyn...",5
1,Project Group 002,"[Tho Huynh, Xiaolei Luo, Anaya Parashar, Emi S...","[A17156565, A15926402, A17210415, A15990255, A...","[thohuynh22, x6luo, None, emisandoval, maddywi...",5
2,Project Group 003,"[Lakshmikethan Bethamcharla, Srivatsa Gangavar...","[A16667125, A16381750, A16681107, A16102256, A...","[Kethan02, sriGang, None, dbinlee3, ethanttlee]",5
3,Project Group 004,"[Jimmy Cho, Harsha Jagarlamudi, Grant Liu, Jen...","[A17230425, A15800225, A15832700, A15401994, A...","[Jimmycho319, hjagarla, None, jvazquezgonzalez...",5
4,Project Group 005,"[Alfonso Alfonso, Bobby Baylon, Hang Liu, Nath...","[A15776381, A17233593, A16487866, A16579484, A...","[UltraViolett28, bobbybeatzdev, HangLiu01, nma...",5
...,...,...,...,...,...
88,Project Group 108,"[Yining Gu, Yuhang Li, Haoyuan Wang, Shuo Wang...","[A17130385, A16526598, A16162507, A16622869, A...","[YGnina, None, LukeWang777, Oooleaf, kdaeve]",5
89,Project Group 110,"[Lingyu Chen, Stefanie Dao, Trinity Gao, Pengx...","[A16530918, A15974899, A16512260, A15432376, A...","[LinYuMachi, StefanieDao, Trinnnn, Primo-mpx, ...",5
90,Project Group 111,"[Gissel Alcaraz-Andrade, Kenneth Casimiro, Sub...","[A17207543, A17132441, A17193360, A16686082, A...","[gkalcaraz, kencasimiro, subashkatel, Zapls, t...",5
91,Project Group 121,"[Carly Freedman, Jonathan Hsu, Moises Llamas, ...","[A17153470, A16445698, A16152385, A16122338, A...","[Carly-Freedman, Jonathaanh, moissess, jeteel0...",5


In [ ]:
#len(Groups[Groups['n_people'] < 5])

In [28]:
# fix up group names to remove project and replace spaces with _ 
Groups['Canvas Group Name'] = Groups['Canvas Group Name'].apply(lambda x: '_'.join(x.split()[1:]) )

In [30]:
Groups.set_index('Canvas Group Name', inplace = True)
Groups

,names,sids,gitids,n_people
Canvas Group Name,,,,
Group_001,"[Zilu Ji, Laurensius Junaidy, Kendra Lee, Thy ...","[A16608450, A16617195, A16282947, A16570913, A...","[LUsamiLU, LaurensiusJunaidy, kkbunny247, thyn...",5
Group_002,"[Tho Huynh, Xiaolei Luo, Anaya Parashar, Emi S...","[A17156565, A15926402, A17210415, A15990255, A...","[thohuynh22, x6luo, None, emisandoval, maddywi...",5
Group_003,"[Lakshmikethan Bethamcharla, Srivatsa Gangavar...","[A16667125, A16381750, A16681107, A16102256, A...","[Kethan02, sriGang, None, dbinlee3, ethanttlee]",5
Group_004,"[Jimmy Cho, Harsha Jagarlamudi, Grant Liu, Jen...","[A17230425, A15800225, A15832700, A15401994, A...","[Jimmycho319, hjagarla, None, jvazquezgonzalez...",5
Group_005,"[Alfonso Alfonso, Bobby Baylon, Hang Liu, Nath...","[A15776381, A17233593, A16487866, A16579484, A...","[UltraViolett28, bobbybeatzdev, HangLiu01, nma...",5
...,...,...,...,...
Group_108,"[Yining Gu, Yuhang Li, Haoyuan Wang, Shuo Wang...","[A17130385, A16526598, A16162507, A16622869, A...","[YGnina, None, LukeWang777, Oooleaf, kdaeve]",5
Group_110,"[Lingyu Chen, Stefanie Dao, Trinity Gao, Pengx...","[A16530918, A15974899, A16512260, A15432376, A...","[LinYuMachi, StefanieDao, Trinnnn, Primo-mpx, ...",5
Group_111,"[Gissel Alcaraz-Andrade, Kenneth Casimiro, Sub...","[A17207543, A17132441, A17193360, A16686082, A...","[gkalcaraz, kencasimiro, subashkatel, Zapls, t...",5


### check for unassigned students (using Canvas gradebook and groups from Canvas)
code for this is already in previous quarter script


**THEORETICALLY** this is not a necessary section anymore, as unassigned students are put in canvas groups before group api calls.  but check anyway! 

In [31]:
# make list of assigned student PIDS
assigned = flatten(Groups['sids'].tolist())

In [32]:
# subtract assigned list from all enrolled
enrolled_sids = enrolled_students['SID'].tolist()
unassigned = set(enrolled_sids) - set(assigned)
error_students = set(assigned) - set(enrolled_sids)
error_students

set()

In [33]:
len(unassigned)

0

In [34]:
enrolled_students[enrolled_students['SID'].isin(unassigned)]

,Student,SID,email,github


In [ ]:
# add unassigned students to existing groups
# Groups 90-92 were randonmly given 5 unassigned students
# Groups 6 and 7 were randomly given 1 unassigned student each

### Create github repos (use previous quarter's script)

OK time to create a github repo for each and every group. Note to do this you need to have gh CLI installed and have authenticated it as your user (see gh auth login documentation)

The process will create one repo for each group from the group_template. Next we invite users to their repo. Then we checkout the project locally, rename the files, push, and delete the local copy

In [35]:
# YOU NEED TO CHANGE THINGS HERE EACH QUARTER!!!
# you'll have to change the name of the quarter in command_add_instructors
# and also in grpsuffix

import os
import glob


command_create = 'gh repo create --template COGS108/group_template -y COGS108/' # + groupname
command_invite = 'gh api -X PUT -f permission=push repos/COGS108/' # + groupname/collaborators/username
command_add_instructors = 'gh api -X PUT -f permission=admin /orgs/COGS108/teams/Staff_Fall2022/repos/COGS108/' # + groupname

grpsuffix = '-Fa22' #change this name suffix each quarter

In [ ]:

for grp in Groups.index:

    gname = grp+grpsuffix 
    
    # create new repo
    cmd = command_create+gname
    ret = os.system(cmd)
    print(cmd,ret)
    
    # add instructors to have admin access to the repo
    cmd=command_add_instructors+gname
    ret = os.system(cmd)
    print(cmd,ret)
    
    # invite everyone in the group to the repo
    for uname in Groups.loc[grp,'gitids']:
        cmd=command_invite+gname+'/collaborators/'+uname.strip()
        ret = os.system(cmd)
        print(cmd,ret)
        
    # get a local copy of the repo
    cmd='gh repo clone COGS108/'+gname
    ret = os.system(cmd)
    print(cmd,ret)

    # rename the files in the repo to the groups name pattern
    for fname in glob.glob(gname+'/*groupXXX.ipynb'):
        source = fname.split('/')[1]
        target = source.split("_")[0]+gname+'.ipynb'
        cmd = ( 'cd '+gname+'; '+
                        ' '.join(['git mv',source,target]))
        ret = os.system(cmd)
        print(cmd,ret)
        
    # commit changes and push
    cmd = 'cd '+gname+'; git add -A; git commit -m "renamed files"; git push'
    ret = os.system(cmd)
    print(cmd,ret)
    
    # remove local copy of repo
    cmd = 'rm -rf '+gname
    ret = os.system(cmd)
    print(cmd,ret)
    
    # add instructors to the repo
    cmd=command_add_instructors+gname
    ret = os.system(cmd)
    print(cmd,ret)

gh repo create --template COGS108/group_template -y COGS108/Group_001-Fa22 256
